In [165]:
import requests
import re 
import os 
import time
import traceback 
from bs4 import BeautifulSoup 
from lxml import etree
from fake_useragent import UserAgent
import pandas as pd

In [166]:
city_url = 'https://raw.githubusercontent.com/ucsb-dreamlab/dianpingBusinessScrape/master/City_URL.csv'
city = pd.read_csv(city_url, index_col= 0)
city.head(10)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
5,安顺,www.dianping.com/anshun/ch75/g34311
6,阿里,www.dianping.com/ali/ch75/g34311
7,安康,www.dianping.com/ankang/ch75/g34311
8,安国市,www.dianping.com/anguo/ch75/g34311
9,安新县,www.dianping.com/anxin/ch75/g34311


Filter this dataset by city levels. Now it has 411 results, and most cities in this filtered dataset are tier 1-3 city. 

In [168]:
filtered_city = city[-(
                city['city_name'].str.endswith('市')| 
                city['city_name'].str.endswith('县')|
                city['city_name'].str.endswith('镇')| 
                city['city_name'].str.endswith('旗')|
                city['city_name'].str.endswith('村')| 
                city['city_name'].str.endswith('区'))].reset_index(drop=True)
display(filtered_city)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
...,...,...
406,诸暨,www.dianping.com/zhuji/ch75/g34311
407,章丘,www.dianping.com/zhangqiu/ch75/g34311
408,周庄,www.dianping.com/zhouzhuang/ch75/g34311
409,朱家尖,www.dianping.com/zhujiajian/ch75/g34311


In [177]:
def getHtmlContent(url): 
 UA = UserAgent(verify_ssl = False)   
 headers = {
    "User-Agent": UA.random
    }

 try:
   s = requests.Session()
   r = s.get(url, headers = headers, cookies = {'from-my':'browser'})
      #r = s.get(url, headers = headers, cookies= s.cookies)
   r.raise_for_status()
   if(r.encoding != "UTF-8"):
      r.encoding = r.apparent_encoding
   return r.text
 except:
   return "Request Error"

In [178]:
URL =  "http://"+ filtered_city['URL'][1] # add "https" to avoid schema error 
URL

'http://www.dianping.com/anshan/ch75/g34311'

In [179]:
HTML = getHtmlContent(URL)

In [173]:
soup = BeautifulSoup(HTML, "html.parser")
print(soup)

<!DOCTYPE html>

<html><head> <meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
<meta content="webkit" name="renderer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<style>
        .container {
            width: 998px;
            margin: 0 auto;
        }
        .header {
            height: 70px;
            font-size: 28px;
            color: #999999;
            border-bottom: 3px solid #FFC300;
            overflow: hidden;
            display: none;
        }
        .logo {
            margin-top: 8px;
            margin-left: 1px;
            height: 54px;
            line-height: 54px;
            padding-left: 160px;
            background: url('https://s3plus.meituan.net/v1/mss_f231eb419c414559a1837748d11d4312/yoda-resources/mtlog.png') no-repeat;
            background-size: auto 54px;
        }
        .dpHeader {
            height: 70px;
            font-size: 28px;
           

In [175]:
# jon gets an error in this cell
# Hi Jon, to excecute this code block, you need to open your browser first. 
# it read the cookies from your browser, otherwise you can't retrieve the html code of dianping webpage.
# (and the above code will return the html code of the verification center(Captcha))
tree = etree.HTML(str(soup))
URLlist = []
for i in range(1, 3):
    link = tree.xpath('//*[@id="shop-all-list"]/ul/li['+ str(i) + ']/div[2]/div[1]/a/@href')
    URLlist.append(link[0])



IndexError: list index out of range

In [19]:
print(URLlist)

['https://www.dianping.com/shop/k7Vqjt0vfkX0HeFA']


//*[@id="shop-all-list"]/ul/li[1]/div[2]/div[1]/a/h4

In [46]:
Liblist = []
for i in range(1,16):
    Libname = tree.xpath('//*[@id="shop-all-list"]/ul/li[' + str(i)+ ']/div[2]/div[1]/a/h4/text()')
    Liblist.append(Libname[0])

In [47]:
print(Liblist)

['100层亲子图书馆', '星星和蒲公英儿童图书馆', '老约翰绘本馆(济南二站)', '倍爱读绘本馆(融创茂店)', '倍爱读绘本馆(领秀城贵和店)', '嘟嘟绘本馆', '老约翰绘本馆', '绘本(振华商厦店)', '读小书儿童绘本馆', '绘本(银座购物广场店)', '共享绘本馆', '山东省少儿图书馆', '济南市图书馆', '山东省图书馆', '共享绘本馆']


In [48]:
JN_lib =pd.DataFrame({'Shop Name':Liblist,'URL': URLlist}) 
JN_lib

,Shop Name,URL
0,100层亲子图书馆,https://www.dianping.com/shop/H9LR34sHvCC9Xn0h
1,星星和蒲公英儿童图书馆,https://www.dianping.com/shop/l6T62dG5LsOFHbjZ
2,老约翰绘本馆(济南二站),https://www.dianping.com/shop/H6LmwseysJyQNvLI
3,倍爱读绘本馆(融创茂店),https://www.dianping.com/shop/l409Jgo7g3DbnkAo
4,倍爱读绘本馆(领秀城贵和店),https://www.dianping.com/shop/k7KfUOpAfQpQDccR
5,嘟嘟绘本馆,https://www.dianping.com/shop/G1PSYhoeegc6Rzzm
6,老约翰绘本馆,https://www.dianping.com/shop/l7vgwCwPFrElKNh5
7,绘本(振华商厦店),https://www.dianping.com/shop/H8ju8cLlBw7jVp6F
8,读小书儿童绘本馆,https://www.dianping.com/shop/G6x135yWr4qCNDBH
9,绘本(银座购物广场店),https://www.dianping.com/shop/k6AArWYyt0kA6Ilm


In [49]:
subpage_soup = []
for i in range(0,len(JN_lib)):
    subpage_soup.append(BeautifulSoup(getHtmlContent(url = JN_lib['URL'][i]), "html.parser"))
    time.sleep(3)

In [60]:
subpage_soup[4]

<!DOCTYPE html>

<html>
<head>
<link href="http://www.dianping.com/shop/k7KfUOpAfQpQDccR" rel="canonical"/>
<script type="text/javascript">(function(){var g=["iPhone","Android","Windows Phone"],b="http://m.dianping.com",e=window.location.pathname+window.location.search,f=false;for(var d=0,a;a=g[d];d++){if(navigator.userAgent.indexOf(a)!=-1){f=true;break}}if(document.referrer.indexOf("baidu.com")>0){if(window.location.search==""){e=e+"?msource=baidusearch"}else{e=e+"&msource=baidusearch"}}else if(document.referrer.indexOf("sogou.com")>0){if(window.location.search==""){e=e+"?msource=sogousearch"}else{e=e+"&msource=sogousearch"}}function h(){location.href=b+e}function c(){if(document.referrer.indexOf(b)==0){document.cookie="vmod=pc;expires=0;"}return document.cookie.match(/vmod=pc/)}if(f){if(!c()){h()}}})();</script>
<title>倍爱读绘本馆（领秀城贵和店）课程_价格_简介_怎么样-济南学习培训-大众点评网</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="pc" name="applicable-device"/>
<meta content="倍爱读

In [50]:

Tree = []
for i in range(0, len(subpage_soup)):
    Tree.append(etree.HTML(str(subpage_soup[i])))
    

In [64]:
Tree[5]

<Element html at 0x25a079289c0>

In [63]:
etree.tostring(Tree[5], pretty_print = True, method = "html")

b'<html>\n<head>\n<link href="http://www.dianping.com/shop/G1PSYhoeegc6Rzzm" rel="canonical">\n<script type="text/javascript">(function(){var g=["iPhone","Android","Windows Phone"],b="http://m.dianping.com",e=window.location.pathname+window.location.search,f=false;for(var d=0,a;a=g[d];d++){if(navigator.userAgent.indexOf(a)!=-1){f=true;break}}if(document.referrer.indexOf("baidu.com")>0){if(window.location.search==""){e=e+"?msource=baidusearch"}else{e=e+"&msource=baidusearch"}}else if(document.referrer.indexOf("sogou.com")>0){if(window.location.search==""){e=e+"?msource=sogousearch"}else{e=e+"&msource=sogousearch"}}function h(){location.href=b+e}function c(){if(document.referrer.indexOf(b)==0){document.cookie="vmod=pc;expires=0;"}return document.cookie.match(/vmod=pc/)}if(f){if(!c()){h()}}})();</script>\n<title>&#22047;&#22047;&#32472;&#26412;&#39302;&#35838;&#31243;_&#20215;&#26684;_&#31616;&#20171;_&#24590;&#20040;&#26679;-&#27982;&#21335;&#23398;&#20064;&#22521;&#35757;-&#22823;&#2024

In [84]:
import numpy as np

In [98]:
AddressList = []
HourList = []
PhoneNumList = []

for i in range(0, len(Tree)):
    try:
        AddressList.append(Tree[i].xpath('/html/body/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[2]/text()[normalize-space()]')[0].strip())
    except IndexError:
        AddressList.append(np.NaN)


In [99]:
for i in range(0, len(Tree)):
    try:
        PhoneNumList.append(Tree[i].xpath('/html/body/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[4]/div/span/text()[normalize-space()]')[0].strip())
    except IndexError:
        PhoneNumList.append(np.NaN)


In [114]:
encrypted_page_index = []
for i in range(0, len(Tree)):
    try:
        HourList.append(Tree[i].xpath('//*[@id="info"]/ul/li[3]/text()[normalize-space()]')[0].strip())
    except IndexError:
        HourList.append(np.NaN)
        encrypted_page_index.append(i+1)
print(encrypted_page_index)

[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [108]:
AddressList

['舜宁路29号',
 '洪山路金邸山庄2号楼底商',
 nan,
 '经十路与凤鸣路交叉口融创茂一楼中厅',
 '领仁路领秀城贵和购物中心一层',
 '解放东路34号现代逸城综合楼1楼',
 '鲁能领秀城中央公园东区商业街16-206室',
 '经四路3号振华商厦（经四路店)F4层',
 '解放路16-3号德冠楼3楼',
 nan,
 '华奥路567号德润天玺12-1-103商铺',
 nan,
 nan,
 nan,
 '花园东路与辛河路交叉口东南200米']

In [109]:
PhoneNumList

['400****096-53762',
 '400****096-89679',
 nan,
 '400****096-47159',
 nan,
 nan,
 '400****096-37219',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [110]:
HourList

['周二至周日\n09:00-21:00',
 '周二至周日\n09:30-21:00',
 nan,
 '周一至周日\n10:00-22:00',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [126]:
Tree[6].xpath('/html/body/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[2]/text()[normalize-space()]')[0].strip()

'鲁能领秀城中央公园东区商业街16-206室'